In [16]:
import numpy as np

In [17]:
params = np.random.uniform(low =-50, high=150, size=20)
params

array([ 44.01835923, -26.73806929,  17.98055763, 134.09318775,
        21.30836871, 133.54857662,  84.82477701,  32.84000663,
       -15.2750435 ,  -1.0422719 , -23.35666243,  70.81090097,
       -28.39405117, 140.30149603, 102.03147933, 117.59158773,
        80.41547862,  46.46386366,  91.47931112,  54.11613651])

In [18]:
params[0] = params.max() + 1
params[1] = params.min() - 1
params[2] = 0

params = np.round(params, 2)
params

array([141.3 , -29.39,   0.  , 134.09,  21.31, 133.55,  84.82,  32.84,
       -15.28,  -1.04, -23.36,  70.81, -28.39, 140.3 , 102.03, 117.59,
        80.42,  46.46,  91.48,  54.12])

In [19]:
def clamp(params_q: np.array, lower_bound: int, upper_bound: int) -> np.array:
    params_q[params_q < lower_bound] = lower_bound
    params_q[params_q > upper_bound] = upper_bound
    return params_q

In [20]:
def asymmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float, int]:
    # Calculate the scale and zero point
    alpha = np.max(params)
    beta = np.min(params)
    scale = (alpha - beta) / (2**bits-1)
    zero = -1*np.round(beta / scale)
    lower_bound, upper_bound = 0, 2**bits-1

    # quantize the parameters
    quantized = clamp(np.array(params / scale + zero), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale, zero

def asymmetric_dequantize(params_q: np.array, scale: float, zero: int) -> np.array:
    return (params_q - zero) * scale



def symmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float]:
    # Calculate the scale
    alpha = np.max(np.abs(params))
    scale = alpha / (2**(bits - 1)-1)
    lower_bound = -2**(bits-1)
    upper_bound = 2**(bits-1)-1
    # Quantize the parameters
    quantized = clamp(np.round(params / scale), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale
    
def symmetric_dequantize(params_q: np.array, scale: float) -> np.array:
    return params_q* scale

In [21]:
def quantization_error(params: np.array, params_q: np.array):
    # Calculate the MSE
    return np.mean((params - params_q)**2)

In [22]:
(asymmetric_q, asymmetric_scale, asymmetric_zero) = asymmetric_quantization(params, 8)
(symmetric_q, symmetric_scale) = symmetric_quantization(params, 8)

print(f'Original: \n{np.round(params, 2)}')

print(f'Asymmetric scale: {asymmetric_scale}, zero: {asymmetric_zero}')
print(asymmetric_q)

print(f'Symmetric scale: {symmetric_scale}')
print(symmetric_q)

Original: 
[141.3  -29.39   0.   134.09  21.31 133.55  84.82  32.84 -15.28  -1.04
 -23.36  70.81 -28.39 140.3  102.03 117.59  80.42  46.46  91.48  54.12]
Asymmetric scale: 0.6693725490196079, zero: 44.0
[255   0  44 244  75 243 170  93  21  42   9 149   1 253 196 219 164 113
 180 124]
Symmetric scale: 1.1125984251968506
[127 -26   0 121  19 120  76  30 -14  -1 -21  64 -26 126  92 106  72  42
  82  49]


In [23]:
# Dequantize the parameters back to float 32 bits
params_deq_asymmetric = asymmetric_dequantize(asymmetric_q, asymmetric_scale, asymmetric_zero)
params_deq_symmetric = symmetric_dequantize(symmetric_q, symmetric_scale)

print(f'Original: \n{np.round(params, 2)}')

print(f'Dequantized asymmetric:')
print(params_deq_asymmetric)

print(f'Dequantized symmetric:')
print(params_deq_symmetric)

Original: 
[141.3  -29.39   0.   134.09  21.31 133.55  84.82  32.84 -15.28  -1.04
 -23.36  70.81 -28.39 140.3  102.03 117.59  80.42  46.46  91.48  54.12]
Dequantized asymmetric:
[141.23760784 -29.45239216   0.         133.8745098   20.75054902
 133.20513725  84.34094118  32.7992549  -15.39556863  -1.3387451
 -23.42803922  70.28411765 -28.78301961 139.89886275 101.74462745
 117.14019608  80.32470588  46.18670588  91.03466667  53.54980392]
Dequantized symmetric:
[141.3        -28.92755906   0.         134.62440945  21.13937008
 133.51181102  84.55748031  33.37795276 -15.57637795  -1.11259843
 -23.36456693  71.20629921 -28.92755906 140.18740157 102.35905512
 117.93543307  80.10708661  46.72913386  91.23307087  54.51732283]


In [24]:
print(f'{"Asymmetric error: ":>20}{np.round(quantization_error(params, params_deq_asymmetric), 2)}')
print(f'{"Symmetric error: ":>20}{np.round(quantization_error(params, params_deq_symmetric), 2)}')

  Asymmetric error: 0.12
   Symmetric error: 0.1
